一种用于度量聚类效果的指标是SSE（sum of Squared Error，误差平方和）。

二分K-Means 算法：
该算法首先将所有的点作为一个簇，然后将该簇一分为二。之后选择其中一个簇继续进行划分。选择哪个簇进行划分取决于对其划分是否可以最大程度降低SSE的值

In [1]:
import numpy as np

In [5]:
def loadDataSet(fileName):
    dataMat =[]
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split("\t")
        fltLine = list(map(float,curLine))
        dataMat.append(fltLine)
    return dataMat

In [2]:
def distEclud(vecA,vecB):
    return np.sqrt(np.sum(np.power(vecA-vecB,2)))

In [3]:
def randCent(dataMat,k):
    n = dataMat.shape[1]
    centroids = np.mat(np.zeros((k,n)))
    for j in range(n):
        minJ = np.min(dataMat[:,j])
        rangJ = float(np.max(dataMat[:,j])- minJ)
        centroids[:,j] = minJ + rangJ * np.random.rand(k,1)
    return centroids

In [16]:
def KMeans(dataMat,k , dist = distEclud,createCent = randCent):
    m = dataMat.shape[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroids = createCent(dataMat,k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = np.inf ; minIndex = -1
            for j in range(k):
                distJI = dist(dataMat[i,:],centroids[j,:])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i,0] != minIndex :
                clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist **2
        print(centroids)
        for cent in range(k):
            ptsInClust = dataMat[np.nonzero(clusterAssment[:,0].A==cent)[0]]
            centroids[cent,:] = np.mean(ptsInClust,axis=0)
        return centroids,clusterAssment

In [17]:
fileName = "testSet.txt"
dataList = loadDataSet(fileName)

In [18]:
dataMat = np.mat(dataList)
myCentroids , clustAssing = KMeans(dataMat,4)

[[ 1.25275957 -3.36686843]
 [-4.58901932  1.73669245]
 [ 4.33748497  2.52882792]
 [-3.30243718  0.11060232]]


In [19]:
def biKMeans(dataMat,k,distMeas = distEclud):
    m = dataMat.shape[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroid0 = np.mean(dataMat,axis=0).tolist()[0]
    centList = [centroid0]
    for j in range(m):
        clusterAssment[j,1] = distMeas(np.mat(centroid0),dataMat[j,:]) **2
    while(len(centList)<k):
        lowestSSE = np.inf
        for i in range(len(centList)):
            ptsInCurrentCluster = dataMat[np.nonzero(clusterAssment[:,0].A==i)[0],:]
            centroidMat,splitClustAss = KMeans(ptsInCurrentCluster,2,distMeas)
            sseSplit = np.sum(splitClustAss[:,1])
            sseNotSplit = np.sum(clusterAssment[np.nonzero(clusterAssment[:,0].A !=i)[0],1])
            print("sseSplit and sseNotSplit :" ,sseSplit,sseNotSplit)
            if(sseSplit + sseNotSplit) < lowestSSE:
                lowestSSE = sseSplit + sseNotSplit
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
        bestClustAss[np.nonzero(bestClustAss[:,0].A== 1)[0],0] = len(centList)
        bestClustAss[np.nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        centList[bestCentToSplit] = bestNewCents[0,:]
        centList.append(bestNewCents[1,:])
        clusterAssment[np.nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
    return centList,clusterAssment

In [20]:
centList,myNewAssment = biKMeans(dataMat,3)

[[-2.3904144  -2.10554694]
 [ 3.68437533  3.40582448]]
sseSplit and sseNotSplit : 1113.07235419 0.0
[[-4.30304081  2.50557118]
 [-4.44330852  2.84961436]]
sseSplit and sseNotSplit : 1936.34287149 243.118628549
[[-0.37868348  0.55177851]
 [ 0.09761848  2.7057615 ]]
sseSplit and sseNotSplit : 296.805885118 869.953725641
